In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/gs건설 프로젝트/stock_news_combine.csv')
df2 = df[['date','거래량','Tokens_2']]
df2

,date,거래량,Tokens_2
0,2021-08-02,491155.0,세상 강일 세상 강일 투시도 고덕 강일 공공 세상 강일 이달 단지 지하 구성 비즈니...
1,2021-08-02,491155.0,동안 아파트 모습 수주 적극 달하 대형 집중 과거 수주 업계 우성 아파트 시공 수주...
2,2021-08-02,491155.0,해외 본드 비용 발생 하반기 실적 개선 기대 건설업 예상 부진 성적표 바이러스 감염...
3,2021-08-02,491155.0,마감 주요 공시 한화 시스템 한화 시스템 연결 기준 영업 이익 전년 동기 증가 잠정...
4,2021-08-02,491155.0,기업 신뢰 조사 자리 되찾 총수 신뢰도 차세대 총수 신뢰도 신세계 부회장 차지 사옥...
...,...,...,...
15724,2023-10-27,371243.0,앵커 강북 단지 오늘 불패 행진 지역 책정 성적표 시장 관심 집중 현장 사업 대어 ...
15725,2023-10-27,371243.0,철근 누락 기술자 부족 복합 결과 토지 공사 사장 오전 국회 국토 교통 위원회 국토...
15726,2023-10-27,371243.0,다양 차별 사회 공헌 활동 선도 기업 발돋움 문화 예술인 학교 체육 지원 사업 커넥...
15727,2023-10-27,371243.0,토지 공사 사장 아파트 철근 누락 사태 설계 검증 기능 약화 관리 감독 문제 복합 ...


In [ ]:
# 'date' 열에서 중복된 값을 찾아서 'date'와 'news_stock' 열을 생성
import pandas as pd

df = df2

result_df = df.drop_duplicates('date').set_index('date')
result_df['news_stock'] = df.groupby('date').size()

# 인덱스를 다시 초기화하여 'date' 열을 컬럼으로 되돌립니다.
result_df.reset_index(inplace=True)

df2 = result_df[['date','거래량','news_stock']]
df2

,date,거래량,news_stock
0,2021-08-02,491155.0,34
1,2021-08-03,460948.0,23
2,2021-08-04,376383.0,21
3,2021-08-05,295712.0,44
4,2021-08-06,343693.0,7
...,...,...,...
543,2023-10-23,678657.0,35
544,2023-10-24,494521.0,36
545,2023-10-25,407073.0,17
546,2023-10-26,387501.0,22


In [ ]:
df2.to_csv('output_file.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:
X = df2[['news_stock']]
y = df2['거래량']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 주식 가격 데이터도 표준화하기 위해 2D 배열로 변환
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)
y_train_scaled = scaler.fit_transform(y_train)
y_test_scaled = scaler.transform(y_test)

In [ ]:
alpha = 0.01  # L1 규제 강도를 조절할 파라미터
lasso_model = Lasso(alpha=alpha)

In [ ]:
lasso_model.fit(X_train_scaled, y_train_scaled)

Lasso(alpha=0.01)

In [ ]:
# 테스트 데이터로 예측
y_pred = lasso_model.predict(X_test_scaled)

# 모델의 성능 평가 (예를 들어, 평균 제곱 오차)
mse = mean_squared_error(y_test_scaled, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1.239314120407885


In [ ]:
from sklearn.metrics import r2_score

# 모델 평가
r2 = r2_score(y_test_scaled, y_pred)
print(f"R-squared: {r2}")

R-squared: 0.09724277569841566


##K폴드

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
X = df2[['거래량']]
y = df2['news_stock']

X = scaler.fit_transform(X)

In [ ]:
# K-fold Cross Validation 설정
num_folds = 5  # 예를 들어, 5-fold 교차 검증
kf = KFold(n_splits=num_folds, shuffle=True, random_state=0)

In [ ]:
# 모델 및 하이퍼파라미터 설정
alpha = 0.01
lasso_model = Lasso(alpha=alpha)

In [ ]:
# 교차 검증 수행
mse_scores = []  # 각 폴드의 MSE를 저장할 리스트
r2_scores = []   # 각 폴드의 R-squared를 저장할 리스트

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    lasso_model.fit(X_train, y_train)
    y_pred = lasso_model.predict(X_test)

    # 모델 성능 평가
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

In [ ]:
# 교차 검증 결과 출력
mean_mse = sum(mse_scores) / len(mse_scores)
mean_r2 = sum(r2_scores) / len(r2_scores)

print(f"Mean Squared Error (MSE) for {num_folds}-fold Cross Validation: {mean_mse}")
print(f"Mean R-squared (R2) for {num_folds}-fold Cross Validation: {mean_r2}")

Mean Squared Error (MSE) for 5-fold Cross Validation: 358.85720593149824
Mean R-squared (R2) for 5-fold Cross Validation: 0.17365320490570257
